# Hello pgvector: Create, store and query OpenAI embeddings in PostgreSQL using pgvector

This notebook will teach you:
- How to create embeddings from content using the OpenAI API
- How to use PostgreSQL as a vector database and store embeddings data in it using pgvector.
- How to use embeddings retrieved from a vector database to augment LLM generation. 

We'll be using the example of creating a chatbot to answer questions about Timescale use cases, referencing content from the Timescale Developer Q+A blog posts. 

This is a great first step to building something like chatbot that can reference a company knowledge base or developer docs.

Let's get started!

Note: This notebook uses a PostgreSQL database with pgvector installed that's hosted on Timescale. You can create your own cloud PostgreSQL database in minutes [at this link](https://console.cloud.timescale.com/signup) to follow along. You can also use a local PostgreSQL database if you prefer.



### Configuration
- Signup for an OpenAI Developer Account and create an API Key. See [OpenAI's developer platform](https://platform.openai.com/overview).
- Install Python
- Install and configure a python virtual environment. We recommend [Pyenv](https://github.com/pyenv/pyenv)
- Install the requirements for this notebook using the following command:

```
pip install -r requirements.txt
```

In [ ]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

In [ ]:
# Run export OPENAI_API_KEY=sk-YOUR_OPENAI_API_KEY...
# Get openAI api key by reading local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key  = os.environ['OPENAI_API_KEY'] 

## Part 1: Create Embeddings
First, we'll create embeddings using the OpenAI API on some text we want to augment our LLM with.
In this example, we'll use content from the Timescale blog about real world use cases.

In [ ]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('blog_posts_data.csv')
df.head()

### 1.1 Calculate cost of embedding data
It's usually a good idea to calculate how much creating embeddings for your selected content will cost.
We use a number of helper functions to calculate a cost estimate before creating the embeddings to help us avoid surprises.

For this toy example, since we're using a small dataset, the total cost will be less than $0.01.

In [ ]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001

# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['content'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

# Helper function: get embeddings for a text
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input = text.replace("\n"," ")
    )
    embedding = response['data'][0]['embedding']
    return embedding

In [ ]:
# quick check on total token amount for price estimation
total_cost = get_total_embeddings_cost()
print("estimated price to embed this content = $" + str(total_cost))

### 1.2 Create smaller chunks of content
The OpenAI API has a limit to the maximum amount of tokens it create create an embedding for in a single request. To get around this limit we'll break up our text into smaller chunks. In general its a best practice to create embeddings of a certain size in order to get better retrieval. For our purposes, we'll aim for chunks of around 512 tokens each.

Note: If you prefer to skip this step, you can use use the provided file: blog_data_and_embeddings.csv which contains the data and embeddings that you'll generate in this step.

In [ ]:
###############################################################################
# Create new list with small content chunks to not hit max token limits
# Note: the maximum number of tokens for a single request is 8191
# https://openai.com/docs/api-reference/requests
###############################################################################
# list for chunked content and embeddings
new_list = []
# Split up the text into token sizes of around 512 tokens
for i in range(len(df.index)):
    text = df['content'][i]
    token_len = num_tokens_from_string(text)
    if token_len <= 512:
        new_list.append([df['title'][i], df['content'][i], df['url'][i], token_len])
    else:
        # add content to the new list in chunks
        start = 0
        ideal_token_size = 512
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_content = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_content = words[start:end]
            new_content_string = ' '.join(new_content)
            new_content_token_len = num_tokens_from_string(new_content_string)
            if new_content_token_len > 0:
                new_list.append([df['title'][i], new_content_string, df['url'][i], new_content_token_len])
            start += ideal_size
            end += ideal_size

In [ ]:
# Create embeddings for each piece of content
for i in range(len(new_list)):
    text = new_list[i][1]
    embedding = get_embeddings(text)
    new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['title', 'content', 'url', 'tokens', 'embeddings'])
df_new.head()

In [ ]:
# Save the dataframe with embeddings as a CSV file
df_new.to_csv('blog_data_and_embeddings.csv', index=False)
# It may also be useful to save as a json file, but we won't use this in the tutorial
#df_new.to_json('blog_data_and_embeddings.json')

## Part 2: Store embeddings with pgvector
In this section, we'll store our embeddings and associated metadata. 

We'll use PostgreSQL as a vector database, with the pgvector extension. 

You can create a cloud PostgreSQL database for free on [Timescale](https://console.cloud.timescale.com/signup) or use a local PostgreSQL database for this step.

### 2.2 Connect to and configure your vector database


In [ ]:
# Timescale database connection string
# Found under "Service URL" of the credential cheat-sheet or "Connection Info" in the Timescale console
# In terminal, run: export TIMESCALE_CONNECTION_STRING=postgres://<fill in here>

connection_string  = os.environ['TIMESCALE_CONNECTION_STRING'] 

In [ ]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(connection_string)
cur = conn.cursor()

#install pgvector 
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

# Register the vector type with psycopg2
register_vector(conn)

# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS embeddings (
            id bigserial primary key, 
            title text,
            url text,
            content text,
            tokens integer,
            embedding vector(1536)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()

Optional: Uncomment and execute the following code only if you need to read the embeddings and metadata from the provided CSV file

In [ ]:
# Uncomment and execute this cell only if you need to read the blog data and embeddings from the provided CSV file
# Otherwise, skip to next cell
'''
df = pd.read_csv('blog_data_and_embeddings.csv')
titles = df['title']
urls = df['url']
contents = df['content']
tokens = df['tokens']
embeds = [list(map(float, ast.literal_eval(embed_str))) for embed_str in df['embeddings']]

df_new = pd.DataFrame({
    'title': titles,
    'url': urls,
    'content': contents,
    'tokens': tokens,
    'embeddings': embeds
})
'''

### 2.3 Ingest and store vector data into PostgreSQL using pgvector
In this section, we'll batch insert our embeddings and metadata into PostgreSQL and also create an index to help speed up search.

In [ ]:
register_vector(conn)
cur = conn.cursor()

In [ ]:
# Remind ourselves of the dataframe structure
df_new.head()

Batch insert embeddings using psycopg2's ```execute_values()```

In [ ]:
#Batch insert embeddings and metadata from dataframe into PostgreSQL database

# Prepare the list of tuples to insert
data_list = [(row['title'], row['url'], row['content'], int(row['tokens']), np.array(row['embeddings'])) for index, row in df_new.iterrows()]
# Use execute_values to perform batch insertion
execute_values(cur, "INSERT INTO embeddings (title, url, content, tokens, embedding) VALUES %s", data_list)
# Commit after we insert all embeddings
conn.commit()

Sanity check by running some simple queries against the embeddings table

In [ ]:
cur.execute("SELECT COUNT(*) as cnt FROM embeddings;")
num_records = cur.fetchone()[0]
print("Number of vector records in table: ", num_records,"\n")
# Correct output should be 129

In [ ]:
# print the first record in the table, for sanity-checking
cur.execute("SELECT * FROM embeddings LIMIT 1;")
records = cur.fetchall()
print("First record in table: ", records)

Create index on embedding column for faster cosine similarity comparison

In [ ]:
# Create an index on the data for faster retrieval
# this isn't really needed for 129 vectors, but it shows the usage for larger datasets
# Note: always create this type of index after you have data already inserted into the DB

#calculate the index parameters according to best practices
num_lists = num_records / 1000
if num_lists < 10:
    num_lists = 10
if num_records > 1000000:
    num_lists = math.sqrt(num_records)

#use the cosine distance measure, which is what we'll later use for querying
cur.execute(f'CREATE INDEX ON embeddings USING ivfflat (embedding vector_cosine_ops) WITH (lists = {num_lists});')
conn.commit() 

## Part 3: Nearest Neighbor Search using pgvector

In this final part of the tutorial, we will query our embeddings table. 

We'll showcase an example of RAG: Retrieval Augmented Generation, where we'll retrieve relevant data from our vector database and give it to the LLM as context to use when it generates a response to a prompt.

In [ ]:
# Helper function: get text completion from OpenAI API
# Note max tokens is 4097
# Note we're using the latest gpt-3.5-turbo-0613 model
def get_completion_from_messages(messages, model="gpt-3.5-turbo-0613", temperature=0, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [ ]:
# Helper function: Get top 3 most similar documents from the database
def get_top3_similar_docs(query_embedding, conn):
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    cur.execute("SELECT content FROM embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

### 3.1 Define a prompt for the LLM
Here we'll define the prompt we want the LLM to provide a reponse to.

We've picked an example relevant to the blog post data stored in the database.

In [ ]:
# Question about Timescale we want the model to answer
input = "How is Timescale used in IoT?"

In [ ]:
# Function to process input with retrieval of most similar documents from the database
def process_input_with_retrieval(user_input):
    delimiter = "```"

    #Step 1: Get documents related to the user input from database
    related_docs = get_top3_similar_docs(get_embeddings(user_input), conn)

    # Step 2: Get completion from OpenAI API
    # Set system message to help set appropriate tone and context for model
    system_message = f"""
    You are a friendly chatbot. \
    You can answer questions about timescaledb, its features and its use cases. \
    You respond in a concise, technically credible tone. \
    """

    # Prepare messages to pass to model
    # We use a delimiter to help the model understand the where the user_input starts and ends
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant Timescale case studies information: \n {related_docs[0][0]} \n {related_docs[1][0]} {related_docs[2][0]}"}   
    ]

    final_response = get_completion_from_messages(messages)
    return final_response

In [ ]:
response = process_input_with_retrieval(input)
print(input)
print(response)

In [ ]:
# We can also ask the model questions about specific documents in the database
input_2 = "Tell me about Edeva and Hopara. How do they use Timescale?"
response_2 = process_input_with_retrieval(input_2)
print(input_2)
print(response_2)